In [1]:
import numpy as np
from datetime import datetime

from strat import RSIRisingFalling
from quantfreedom.enums import CandleBodyType

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    since_datetime=datetime(2024, 2, 4),
    until_datetime=datetime(2024, 2, 15),
    # candles_to_dl=3000,
)
timestamps = candles[:, CandleBodyType.Timestamp]

In [3]:
timestamps

array([1707004800000.00, 1707005100000.00, 1707005400000.00, ...,
       1707954300000.00, 1707954600000.00, 1707954900000.00])

In [4]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(30, 41, 5),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_strat.plot_signals(candles=candles)

In [5]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1, 0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [6]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 3
Total order settings to test: 16
Total combinations of settings to test: 48
Total candles: 3,168
Total candles to test: 152,064


In [7]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
36,2,4,14.0,8,6,101.176,57.143,0.880,228.894,2011.757,1011.757
37,2,5,14.0,8,6,101.176,57.143,0.880,228.894,2011.757,1011.757
39,2,7,14.0,8,6,93.058,57.143,0.872,217.864,1930.578,930.578
38,2,6,14.0,8,6,93.058,57.143,0.872,217.864,1930.578,930.578
45,2,13,14.0,8,6,172.911,57.143,0.839,246.047,2729.107,1729.107
44,2,12,14.0,8,6,172.911,57.143,0.839,246.047,2729.107,1729.107
47,2,15,14.0,8,6,163.465,57.143,0.833,234.032,2634.646,1634.646
46,2,14,14.0,8,6,163.465,57.143,0.833,234.032,2634.646,1634.646
31,1,15,10.0,5,5,96.473,50.000,0.806,93.063,1964.727,964.727
29,1,13,10.0,5,5,96.445,50.000,0.806,93.649,1964.447,964.447


In [8]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    ind_set_index=5,
    dos_index=22,
    plot_results=True,
    logger_bool=False,
)

Exception: Exception long_set_entries_exits_array -> index 5 is out of bounds for axis 0 with size 3

In [ ]:
order_records_df

In [ ]:
order_records_df[order_records_df["order_status"] == "TakeProfitFilled"]

In [ ]:
entry_filled_df = order_records_df[order_records_df["order_status"] == "EntryFilled"]
entry_prices = entry_filled_df[["timestamp", "entry_price"]].to_numpy()
sl_prices = entry_filled_df[["timestamp", "sl_price"]].to_numpy()
tp_prices = entry_filled_df[["timestamp", "tp_price"]].to_numpy()

sl_exits = order_records_df[order_records_df["order_status"] == "StopLossFilled"][
    ["timestamp", "exit_price"]
].to_numpy()
tp_exits = order_records_df[order_records_df["order_status"] == "TakeProfitFilled"][
    ["timestamp", "exit_price"]
].to_numpy()

MovedTSL_prices = order_records_df[order_records_df["order_status"] == "MovedTSL"][["timestamp", "sl_price"]].to_numpy()

In [ ]:
entry_prices_array = np.full((timestamps.size, 2), np.nan)
entry_prices_array[:, 0] = timestamps
entry_prices_array[:, 1] = -1

sl_prices_array = np.full((timestamps.size, 2), np.nan)
sl_prices_array[:, 0] = timestamps
sl_prices_array[:, 1] = -1

tp_prices_array = np.full((timestamps.size, 2), np.nan)
tp_prices_array[:, 0] = timestamps
tp_prices_array[:, 1] = -1

counter = 0

array_size = entry_prices.shape[0]
for idx, timestamp in enumerate(timestamps):
    if entry_prices[counter, 0] == timestamp:
        entry_prices_array[idx, 1] = entry_prices[counter, 1]
        sl_prices_array[idx, 1] = sl_prices[counter, 1]
        tp_prices_array[idx, 1] = tp_prices[counter, 1]
        counter += 1
        if counter >= array_size:
            break

entry_prices_array_list = entry_prices_array.tolist()
sl_prices_array_list = sl_prices_array.tolist()
tp_prices_array_list = tp_prices_array.tolist()

In [ ]:
sl_exits_array = np.full((timestamps.size, 2), np.nan)
sl_exits_array[:, 0] = timestamps
sl_exits_array[:, 1] = -1

counter = 0

array_size = sl_exits.shape[0]
for idx, timestamp in enumerate(timestamps):
    if sl_exits[counter, 0] == timestamp:
        sl_exits_array[idx, 1] = sl_exits[counter, 1]
        counter += 1
        if counter >= array_size:
            break
sl_exits_array_list = sl_exits_array.tolist()

In [ ]:
tp_exits_array = np.full((timestamps.size, 2), np.nan)
tp_exits_array[:, 0] = timestamps
tp_exits_array[:, 1] = -1

counter = 0

array_size = tp_exits.shape[0]
for idx, timestamp in enumerate(timestamps):
    if sl_exits[counter, 0] == timestamp:
        tp_exits_array[idx, 1] = tp_exits[counter, 1]
        counter += 1
        if counter >= array_size:
            break
tp_exits_array_list = tp_exits_array.tolist()

In [ ]:
MovedTSL_prices_array = np.full((timestamps.size, 2), np.nan)
MovedTSL_prices_array[:, 0] = timestamps
MovedTSL_prices_array[:, 1] = -1

counter = 0

array_size = sl_exits.shape[0]
for idx, timestamp in enumerate(timestamps):
    if sl_exits[counter, 0] == timestamp:
        MovedTSL_prices_array[idx, 1] = MovedTSL_prices[counter, 1]
        counter += 1
        if counter >= array_size:
            break
MovedTSL_prices_array_list = MovedTSL_prices_array.tolist()

In [ ]:
entry_prices_array_list
tp_prices_array_list
sl_prices_array_list
sl_exits_array_list
tp_exits_array_list
MovedTSL_prices_array_list